# INSSEPT Interactive Query Tool
This Jupyter notebook is provided as part of INSSEPT database to aid with easy querying and visualization based on site and number of stories. 
Users are kindly requested to acknowledge the use of this software and provides the following citations:
Zaker Esteghamati, M.,Lee, J.,Musetich, M., Flint, M.M., (2019)_INSSEPT: An open-source relational database of seismic performance estimation to aid with early design of buildings_,Submitted to EarthquakeSpectra 
* Questions and comments are encouraged and can be directed to mohsenzaker@vt.edu (Mohsen Zaker Esteghamati)

In [3]:
# importing necessary libraries
import pymysql
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widget
from IPython.display import  clear_output
import math 

# connecting to database
conn= pymysql.connect(host='129.114.58.189', port=3306, user='inssept_read', passwd='INSSEPT_V0.0.1', db='INSSEPT')
sqlCom= """SELECT 
MATSYS.LatSys,BUILDING.Stories, BUILDING.FlrArea, BUILDING.AreaUnits, SITE.Location, SITE.SiteClass, 
INDVAR.IndVar, INDVAR.IndUnits,
DEPVAR.DepVar, DEPVAR.DepUnits,
RESULTS.Parameter, RESULTS.Value, RESULTS.ResRmrk
FROM BUILDING
INNER JOIN MATSYS ON BUILDING.MatSysID=MATSYS.MatSysID
INNER JOIN SITE    ON BUILDING.SiteID = SITE.SiteID
INNER JOIN RESULTS ON BUILDING.BldgID = RESULTS.BldgID
INNER JOIN INDVAR  ON INDVAR.IndVarID = RESULTS.IndVarID
INNER JOIN DEPVAR  ON DEPVAR.DepVarID = RESULTS.DepVarID"""
data=pd.read_sql_query(sqlCom,conn)


def uniqueVal(array):
    unique=array.unique().tolist()
    unique.insert(0,'ALL')
    return unique
    
output=widget.Output()
Visual=widget.Output()
Location_list=uniqueVal(data.Location)
Location_list.remove('CA')
Location_list.remove(None)
dropdown_Location=widget.Dropdown(options=Location_list,description='Location:')
dropdown_Results=widget.Dropdown(options=uniqueVal(data.DepVar),description='Results:')
slider_Stories=widget.IntRangeSlider(value=[3,6],min=1, max=20, step=1,orientation='Horizontal', description='Min Story No.')


def CombinedFilter (Location,Story,DepVar):
    output.clear_output()
    Visual.clear_output()
    if (Location=='ALL') & (DepVar=='ALL'): 
        combined_filter=data[(data.Stories >= Story[0]) & (data.Stories <= Story[1])]
    elif (Location=='ALL'):
         combined_filter=data[(data.DepVar==DepVar) & (data.Stories >= Story[0]) & (data.Stories <= Story[1])]
    elif (Location=='California'):
         if (DepVar=='ALL'): 
            combined_filter=data[((data.Location=='CA') | (data.Location=='California')) & (data.Stories >= Story[0]) & (data.Stories <= Story[1])]
         else:
            combined_filter=data[((data.Location=='CA') | (data.Location=='California')) &(data.DepVar==DepVar) & (data.Stories >= Story[0]) & (data.Stories <= Story[1])]
    elif (DepVar=='ALL'):
          combined_filter=data[(data.Location==Location) & (data.Stories >= Story[0]) & (data.Stories <= Story[1])]
    else:
         combined_filter=data[(data.DepVar==DepVar) & (data.Location==Location) & (data.Stories >= Story[0]) & (data.Stories <= Story[1])]
    with output:
        display(combined_filter)
    with Visual:
      Storymax=max(combined_filter['Stories'])
      Units=data.DepUnits[0]
      xticks_plot1=range(0,Storymax+2)
      plot1=sns.scatterplot(x=combined_filter['Stories'],y=combined_filter['Value'],
                           hue=combined_filter['LatSys'])   
      plot1.set(xticklabels=xticks_plot1)
      plot1.set(xticks=xticks_plot1)
      plot1.set(xlabel='Story Nnumber', ylabel='%s'%(DepVar))
      plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
      plt.show() 
      
      flagArea=0
      for check in combined_filter['FlrArea']:
         if not math.isnan(check):
            flagArea=1
            
      if flagArea:  
          plot2=sns.scatterplot(x=combined_filter['FlrArea'],
                            y=combined_filter['Value'],hue=combined_filter['LatSys'])
          AreaMin=min(combined_filter['FlrArea'])
          AreaMax=max(combined_filter['FlrArea'])
          xticks_plot2=[int(AreaMin/2),int(AreaMin),int(1.5*AreaMax)]
          plot2.set(xticklabels=xticks_plot2)
          plot2.set(xticks=xticks_plot2)
          plot2.set(xlabel='Floor Area', ylabel='%s'%(DepVar))
          plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
          plt.show() 
def dropdown_Location_eventhandler(change):
    CombinedFilter(change.new,slider_Stories.value,dropdown_Results.value)
def slider_Stories_eventhandler(change):
    CombinedFilter(dropdown_Location.value,change.new,dropdown_Results.value)
def dropdown_Results_eventhandler(change):
    CombinedFilter(dropdown_Location.value,slider_Stories.value,change.new)
 
dropdown_Location.observe(dropdown_Location_eventhandler,names='value')
slider_Stories.observe(slider_Stories_eventhandler,names='value')
dropdown_Results.observe(dropdown_Results_eventhandler,names='value')

Margin=widget.Layout(margin='10px 5px 10px 5px')
Layout_widgets= widget.HBox([dropdown_Location,slider_Stories,dropdown_Results],layout=Margin)
Layout_output=widget.Tab([output,Visual],layout=Margin)
Layout_output.set_title(0,'Raw Data')
Layout_output.set_title(1,'Visualization')
LayoutFull=widget.VBox([Layout_widgets,Layout_output])
display(LayoutFull)
